In [72]:
import math
import json
import copy
from typing import List
from collections import defaultdict


In [48]:
def euclid(a,b):
    return math.sqrt((a.x-b.x)**2 + (a.y-b.y)**2)

def manhattan(a,b):
    return abs(a.x-b.x)+bs(a.y-b.y)

def chebyshev(a,b):
    return max(abs(a.x - b.x), abs(a.y - b.y))

In [49]:
##LOJDOVA SLAGALICA

In [50]:
start = [
    [4,1,3],
    [0,2,5],
    [7,8,6]
]
target = [
    [1,2,3],
    [4,5,6],
    [7,8,0]
]

In [51]:
##Funkcije za serijalizaciju i deserijalizaciju
def serialize(matrix):
    return json.dumps(matrix)

def deserialize(value:str):
    return json.loads(value)

In [52]:
##Funkcija za nalazenje target polja
def find_in_matrix(value, matrix):
    n = len(matrix)
    m = len(matrix[0])
    for i in range(n):
        for j in range(m):
            if value == matrix[i][j]:
                return i, j
    return None, None

In [53]:
##OVO SE PISE
def manhattan_sum(trenutni_node, target_node):
    A = deserialize(trenutni_node)
    B = deserialize(target_node)
    
    n = len(A)
    H = 0
    
    for i in range(n):
        for j in range(n):
            value = A[i][j]
            target_i, target_j = find_in_matrix(value, B)
            H += abs(i - target_i) + abs(j - target_j)
    return H

In [54]:
class Edge:
    def __init__(self, start, end, weight:float):
        self.start = start
        self.end = end
        self.weight = weight
        

In [55]:
##PISE SE
def iterate_edges(state:str):
    matrix = deserialize(state)
    blank_i, blank_j = -1, -1
    
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            if matrix[i][j] == 0:
                blank_i, blank_j = i, j
                break
    neighbours = []
    if blank_i > 0:
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i - 1][blank_j]
        new_matrix[blank_i - 1][blank_j] = 0
        neighbours.append(Edge(matrix, serialize(new_matrix), 1))
        
    if blank_i < (n-1):
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i + 1][blank_j]
        new_matrix[blank_i + 1][blank_j] = 0
        neighbours.append(Edge(matrix, serialize(new_matrix), 1))
    
    if blank_j > 0:
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i][blank_j - 1]
        new_matrix[blank_i][blank_j - 1] = 0
        neighbours.append(Edge(matrix, serialize(new_matrix), 1))
    
    if blank_j < (n-1):
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i][blank_j + 1]
        new_matrix[blank_i][blank_j + 1] = 0
        neighbours.append(Edge(matrix, serialize(new_matrix), 1))
    
    return neighbours

In [56]:
##PISE SE
def astar_loyd(start, end, h):
    open_set, closed_set = set([start]), set()
    
    cheapest_paths = defaultdict (lambda: float('inf'))
    
    parent = {start: None}
    
    def get_best():
        result, result_value = None, float('inf')
        for node in open_set:
            new_value = cheapest_paths[node] + h(node, end)
            if not result or new_value < result_value:
                result, result_value = node, new_value
        return result
    
    path_found = False
    while len(open_set) > 0:
        current = get_best()
        if current == end:
            path_found = True
            break
        for edge in iterate_edges(current):
            neighbour = edge.end
            new_path_to_neighbour_weight = cheapest_paths[current] + edge.weight
            if neighbour not in open_set and neighbour not in closed_set:
                open_set.add(neighbour)
                parent[neighbour] = current
                cheapest_paths[neighbour] = new_path_to_neighbour_weight
            elif new_path_to_neighbour_weight < cheapest_paths[neighbour]:
                parent[neighbour] = current
                cheapest_paths[neighbour] = new_path_to_neighbour_weight
                if neighbour in closed_set:
                    closed_set.remove(neighbour)
                    open_set.add(neighbour)
        open_set.remove(current)
        closed_set.add(current)
    if path_found:
        path = []
        while end is not None:
            path.append(end)
            end = parent[end]
        path.reverse()
        return path
    else:
        return None

In [57]:
 astar_loyd(serialize(start), serialize(target), manhattan_sum)

['[[4, 1, 3], [0, 2, 5], [7, 8, 6]]',
 '[[0, 1, 3], [4, 2, 5], [7, 8, 6]]',
 '[[1, 0, 3], [4, 2, 5], [7, 8, 6]]',
 '[[1, 2, 3], [4, 0, 5], [7, 8, 6]]',
 '[[1, 2, 3], [4, 5, 0], [7, 8, 6]]',
 '[[1, 2, 3], [4, 5, 6], [7, 8, 0]]']

In [89]:
##LAVIRINT














In [4]:
maze_matrix = [['S', '#', '.', '.', 'X', '#', '#', '.'],
               ['.', 'X', '#', '.', 'X', '#', '#', '.'],
               ['.', '.', '.', '.', '.', '#', 'X', '.'],
               ['.', 'X', 'X', '#', 'X', '.', 'X', '.'],
               ['#', '.', '.', '.', 'X', '.', 'X', '.'],
               ['.', 'X', '#', '#', '.', '#', '.', '.'],
               ['.', '#', '.', '.', 'X', '.', '#', '.'],
               ['.', '#', '.', '.', 'X', '#', '#', 'F']]

In [5]:
class Maze:
    def __init__(self, maze_matrix):
        self.start, self.finish, self.G = self.convert_matrix_to_graph(maze_matrix)
        self.maze_matrix = maze_matrix
    
    def convert_matrix_to_graph(self, maze_matrix):
        G = {}
        start = None
        finish = None
        n = len(maze_matrix)
        m = len(maze_matrix[0])
        
        for i in range(n):
            for j in range(m):
                v = (i, j)
                neighbors = []
                if maze_matrix[i][j] != 'X':
                    if maze_matrix[i][j] == 'S':
                        start = v
                    if maze_matrix[i][j] == 'F':
                        finish = v
                    if i > 0:
                        if maze_matrix[i-1][j] != 'X':
                            w = (i-1, j)
                            if maze_matrix[i-1][j] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                    if i < n-1:
                        if maze_matrix[i+1][j] != 'X':
                            w = (i+1, j)
                            if maze_matrix[i+1][j] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                    
                    if j > 0:
                        if maze_matrix[i][j-1] != 'X':
                            w = (i, j-1)
                            if maze_matrix[i][j-1] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                            
                    if j < m-1:
                        if maze_matrix[i][j+1] != 'X':
                            w = (i, j+1)
                            if maze_matrix[i][j+1] == '#':
                                weight = 2
                            else:
                                weight = 1
                            neighbors.append((w, weight))
                G[v] = neighbors
        return start, finish, G
    
    def h(self, v_coords):
        v_x1 = int(v_coords[0])
        v_y1 = int(v_coords[1])
        finish_x1 = int(self.finish[0])
        finish_y1 = int(self.finish[1])
        
        return abs(v_x1 - finish_x1) + abs(v_y1 - finish_y1)
    
    def astar(self, start, stop):
        open_list = set([start])
        closed_list = set([])
        
        g = {}
        g[start] = 0
        parents = {}
        parents[start] = None
        
        while len(open_list) > 0:
            n = None
            for v in open_list:
                if n == None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v
            if n == stop:
                path = []
                path.append(stop)
                tmp = parents[stop]
                while tmp != None:
                    path.append(tmp)
                    tmp = parents[tmp]

                path.reverse()
                return path
            
            for m, weight in self.G[n]:
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)
            open_list.remove(n)
            closed_list.add(n)
        
        return None
    
    def solve(self):
        path = self.astar(self.start, self.finish)
        return path
    

In [6]:
maze = Maze(maze_matrix)

In [8]:
print(maze.solve())

[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7)]
